In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from bz2 import BZ2File
import json
import os
from pathlib import Path
import pickle

In [63]:
def _read_write_args(path, mode):
    """Helper for `save` and `load` functions.
    
    Parameters
    ----------
    path: str
        Path to read/write object from/to.
    mode: str
        'w' for writing files (as in `save`), 'r' for reading files 
        (as in `load`).
    
    Returns
    -------
    tuple: Function to open file, mode to open file with (str), object to open
        file with.
    """
    ext = path.rpartition('.')[-1]
    if ext not in {'json', 'pkl', 'zip'}:
        raise InvalidArgumentError(
            'Invalid extension. Make sure your filename ends with .json, '
            '.pkl, or .zip.'
        )
        
    # Store in dict to make it easier to add additional formats in future.
    ext2data = {'pkl': (open, 'b', pickle), 
                'zip': (BZ2File, '', pickle), 
                'json': (open, '', json)}
    opener, mode_suffix, saver = ext2data[ext]
    return opener, mode + mode_suffix, saver


def save(obj, path, verbose=True):
    """Wrapper to save data as pickle (optionally zipped) or json.

    Parameters
    -----------
    obj: any
        Object to save. This will be pickled/jsonified/zipped inside the
        function - do not convert it before-hand.
    path: str
        File name to save object to. Should end with .pkl, .zip, or
        .json depending on desired output format. If .zip is used, object will
        be zipped and then pickled.
    verbose: bool
        If True, print a message confirming that the data was pickled, along
        with its path.

    Returns
    -------
    None
    """
    path = Path(path)
    os.makedirs(path.parent, exist_ok=True)
    if verbose: print(f'Writing data to {path}.')
    if path.suffix == '.txt':
        path.write_text(obj)
    else:
        opener, mode, saver = _read_write_args(str(path), 'w')
        with opener(path, mode) as f:
            saver.dump(obj, f)

def load(path, verbose=True):
    """Wrapper to load pickled (optionally zipped) or json data.
    
    Parameters
    ----------
    path : str
        File to load. File type will be inferred from extension.
    verbose : bool, optional
        If True, will print message stating where object was loaded from.
    
    Returns
    -------
    object: The Python object that was pickled to the specified file.
    """
    path = Path(path)
    if path.suffix == '.txt':
        return path.read_text()
    
    opener, mode, saver = _read_write_args(str(path), 'r')
    with opener(path, mode) as f:
        data = saver.load(f)
    if verbose: print(f'Object loaded from {path}.')
    return data

In [45]:
d = dict(a=3, c=5, e=7)
a = [4, 33, 27, 0]

In [46]:
!pwd
!ls ..

/Users/hmamin/pythonhm/htools/notebooks
LICENSE.txt      data             htools.egg-info  setup.py
MANIFEST.in      dist             notebooks
README.md        docs             requirements.txt
__pycache__      htools           setup.cfg


In [47]:
path = os.path.join('..', 'data', 'new', 'd.zip')
ppath = Path(path.replace('d.', 'e.'))
path, ppath

('../data/new/d.zip', PosixPath('../data/new/e.zip'))

In [48]:
save(d, path)

Writing data to ../data/new/d.zip.


In [49]:
save(a, ppath)

Writing data to ../data/new/e.zip.


In [50]:
!ls ../data/new

d.zip  d3.zip e.zip


In [51]:
load(path)

Object loaded from ../data/new/d.zip.


{'a': 3, 'c': 5, 'e': 7}

In [52]:
load(ppath)

Object loaded from ../data/new/e.zip.


[4, 33, 27, 0]

In [53]:
path = os.path.join('..', 'data', 'a.txt')
ppath = Path(path.replace('a.', 'b.'))
path, ppath

('../data/a.txt', PosixPath('../data/b.txt'))

In [55]:
save(str(a), path)

Writing data to ../data/a.txt.


In [56]:
save(str(d), ppath)

Writing data to ../data/b.txt.


In [61]:
load(path)

'[4, 33, 27, 0]'

In [62]:
load(ppath)

"{'a': 3, 'c': 5, 'e': 7}"